In [ ]:
# the regulars
import pandas as pd
import geopandas as gpd

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
lat =  39.444012
lon = -121.833619

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2018-10-01'

# end date
end_date = '2019-01-31'

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [ ]:
print('Total number:', landsat.size().getInfo())

In [ ]:
#information about the firsr image in our collection
landsat.first().getInfo()

In [ ]:
#what about cloud of our first image ?
landsat.first().get('CLOUD_COVER').getInfo()

In [ ]:
# when was this image taken ? 
landsat.first().get('DATE_ACQUIRED').getInfo()

In [ ]:
# what bands did we get ?
landsat.first().bandNames().getInfo()

In [ ]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

In [ ]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [ ]:
#create an empty data container 
data = []

# loop through each image and display it
for i in range(landsat.size().getInfo()):
    
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    #cloudcover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    #print the image info
    print('Image #',i,date,'Cloud cover:',cloud)
    
    #display the image 
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))
    
    
    #data to list 
    this_data = [i,date,cloud]
    
    # append the data
    data.append(this_data)

 #create the pandas DataFrame

df = pd.DataFrame(data,columns=['Image #','Date','Cloud Cover'])

In [ ]:
#our data in a frame

df

In [ ]:
# create a list of images we want (before,during,after)
landsat_sequence = [0,2,5]

In [ ]:
# Define a region of interest with a buffer zone of 20 km 
roi = poi.buffer(20000) #meters

In [ ]:
parameters = {
              'min':6000,
              'max':16000,
              'dimensions':800,
              'bands': ['SR_B4','SR_B3','SR_B2'],
              'region':roi
    
             }

In [ ]:
for i in landsat_sequence:
    #when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    #cloudcover 
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print ('Image #',i,date,'Cloud cover',cloud)
    display (Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

In [ ]:
# ndvi palette : red is low,green is high vegetation
palette = ['red','yellow','green']

ndvi_parameters = {'min':0,
                  'max':0.4,
                  'dimensions':512,
                  'palette':palette,
                  'region':roi}

In [ ]:
for i in landsat_sequence:
    #when was this image taken ?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    #print some information 
    print('Image #',i,date)
    
    #display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))
                  

In [ ]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])
m

In [ ]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

In [ ]:
my_map.save('camp.html')